# Descripcion del proyecto

El coronavirus tomó al mundo entero por sorpresa, cambiando la rutina y hábitos diarios de las personas. Los habitantes de las ciudades ya no pasaban su tiempo libre fuera yendo a cafés y centros comerciales sino que comenzaron a desarrollar nuevas actividades en sus casas, entre ellas la lectura. Eso atrajo la atención de las startups que se apresuraron a desarrollar nuevas aplicaciones para los amantes de los libros. Es así que se trabajara con esta base de datos de uno de los servicios que compiten en el mercado para generar una propuesta de valor para un nuevo producto


# Descripcion de los datos

# books:

Contiene datos sobre libros:
- book_id — identificación del libro
- author_id — identificación del autor o autora
- title — título
- num_pages — número de páginas
- publication_date — fecha de la publicación
- publisher_id — identificación de la editorial

# **authors:**
Contiene datos sobre autores:
- author_id — identificación del autor o autora
- author — el autor o la autora

# **publishers:**
Contiene datos sobre editoriales:
- publisher_id — identificación de la editorial
- publisher — la editorial


# **ratings:**
Contiene datos sobre las calificaciones de usuarios:
- rating_id — identificación de la calificación
- book_id — identificación del libro
- username — el nombre del usuario que revisó el libro
- rating — calificación


# **reviews:**
Contiene datos sobre las reseñas de los y las clientes:

- review_id — identificación de la reseña
- book_id — identificación del libro
- username — el nombre del usuario que revisó el libro
- text — el texto de la reseña



# Importación librerias y conexion base de datos

In [1]:
#importar librerías
import pandas as pd
from sqlalchemy import create_engine
db_config = {'user': 'practicum_student', # nombre de usuario
'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # puerto de conexión
'db': 'data-analyst-final-project-db'} # nombre de la base de datos
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

ModuleNotFoundError: No module named 'psycopg2'

# Estudio de tablas

# books

In [ ]:

query= ''' select *
           from books limit 5'''

books= pd.io.sql.read_sql(query, con = engine)
display(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


# authors

In [ ]:
query= ''' select *
           from authors limit 5'''


authors = pd.io.sql.read_sql(query, con = engine)
display(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


# publishers

In [ ]:
query= ''' select *
           from publishers limit 5'''


publishers = pd.io.sql.read_sql(query, con = engine)
display(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


# ratings

In [ ]:
query= ''' select *
           from ratings limit 5'''


ratings = pd.io.sql.read_sql(query, con = engine)
display(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


# reviews

In [ ]:
query= ''' select *
           from reviews limit 5'''


reviews = pd.io.sql.read_sql(query, con = engine)
display(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# Número de libros publicados después del 1 de enero de 2000.

In [ ]:
query= '''select count(book_id) as cantidad_de_libros
          from books
          where publication_date > '2000-01-1'
'''


libros_2000 = pd.io.sql.read_sql(query, con = engine)
display(libros_2000)

,cantidad_de_libros
0,819


La cantidad de libros publicados a partir del 1 de enero del 2000 es de 819

#  Número de reseñas de usuarios y la calificación promedio para cada libro.

In [ ]:
query= ''' select books.book_id,
                  authors.author,
                  books.title,
                  count(distinct reviews.review_id) as review,
                  avg(ratings.rating) as avg_rating 
           from books
                left join authors on books.author_id = authors.author_id
                left join reviews on reviews.book_id = books.book_id
                left join ratings on ratings.book_id = books.book_id
          group by books.book_id, authors.author_id'''


reviews = pd.io.sql.read_sql(query, con = engine)
reviews.head()

,book_id,author,title,review,avg_rating
0,1,Stephen King/Jerry N. Uelsmann,'Salem's Lot,2,3.666667
1,2,Patricia Schultz,1 000 Places to See Before You Die,1,2.500000
2,3,Maureen Johnson,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,Charles C. Mann,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,David McCullough,1776,4,4.000000


# Editorial que ha publicado el mayor número de libros con más de 50 páginas.

In [ ]:
query= ''' 
               select publishers.publisher_id,
                      publishers.publisher,
                      count(books.title) as number_books
                from publishers
                  left join books on books.publisher_id = publishers.publisher_id
                where num_pages > 50
                group by publishers.publisher_id
                order by number_books desc
                limit 1
'''


editorial = pd.io.sql.read_sql(query, con = engine)
display(editorial)


,publisher_id,publisher,number_books
0,212,Penguin Books,42


La editorial Penguin books fue la que mas publico libros mayor a 50 paginas siendo 42 libros

# Autor que tiene la más alta calificación promedio : solo los libros con al menos 50 calificaciones.

In [ ]:
query= '''select l.author_id,
                 l.author,
                 avg(ratings.rating) as avg_rating                
          from(select books.author_id,
                      authors.author,
                      ratings.book_id,
                      count(ratings.rating) as number_of_ratings
               from ratings
                  left join books on books.book_id = ratings.book_id
                  inner join authors on books.author_id = authors.author_id
               group by ratings.book_id,
                        books.author_id,
                        authors.author
               having count(ratings.rating) >= 50) as l
             left join ratings on l.book_id = ratings.book_id
         group by  l.author_id,
                   l.author
         order by avg_rating desc
         limit 1
'''

rating = pd.io.sql.read_sql(query, con = engine)
display(rating)

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.287097


Los autores con mayor calificación promedio fue J.K. Rowling/Mary GrandPré con 4.29

#  Número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

In [ ]:
query= ''' select avg(text) as avg_text_review   
           from (select a.username,                         
                        count(reviews.text) as text
                 from (select username,                      
                              count(book_id) as graded_books
                       from ratings
                       group by username
                       having count(book_id) > 50) as a
                    left join reviews on reviews.username= a.username
                group by a.username) as user_reviews
'''


reseñas = pd.io.sql.read_sql(query, con = engine)
display(reseñas)

,avg_text_review
0,24.333333


El promedio de reseñas de los usuarios que califican mas de 50 libros es de 24 reseñas